In [1]:
import pandas as pd

In [74]:
data = pd.read_csv("rsmp/csv/data_product.csv", dtype=str)

In [75]:
data.head(3)

,id,tin,reg_number,kind,category,first_name,last_name,patronymic,org_name,org_short_name,...,region,area,settlement,settlement_type,oktmo,lat,lon,address_raw,start_date,end_date
0,0,6910020514,1116910001669,0,0,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЦЕНТ...","ООО ""ЦЮУ ""ПРИОРИТЕТ""",...,Тверская область,NaN,Кимры,г,28726000001.0,56.8733213,37.3556605,ТВЕРСКАЯ / ОБЛАСТЬ / / / КИМРЫ / ГОРОД / /,2020-11-10,2020-11-10
1,1,0273080245,1100280033897,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АВАН...","ООО ""АВАНТАЖ""",...,Республика Башкортостан,NaN,Уфа,г,80701000001.0,54.734944,55.9578468,БАШКОРТОСТАН / РЕСПУБЛИКА / / / УФА / ГОРОД ...,2017-04-10,2023-07-10
2,2,0276115295,1080276003026,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АКЦЕНТ""","ООО ""АКЦЕНТ""",...,Республика Башкортостан,NaN,Уфа,г,80701000001.0,54.734944,55.9578468,БАШКОРТОСТАН / РЕСПУБЛИКА / / / УФА / ГОРОД ...,2016-12-10,2023-07-10


In [76]:
data["start_date"] = pd.to_datetime(data["start_date"])
data["end_date"] = pd.to_datetime(data["end_date"])

In [77]:
start_year = data["start_date"].dt.year.min()
end_year = data["end_date"].dt.year.max()

In [78]:
panel_elems = []
for year in range(start_year, end_year + 1):
    start_dt = pd.Timestamp(f"{year}-01-01")
    end_dt = pd.Timestamp(f"{year}-12-31")
    yearly_data = data.sort_values("start_date").loc[~((data["start_date"] > end_dt) | (data["end_date"] < start_dt))].copy()
    group_sizes = yearly_data.groupby("tin", as_index=False).size()
    yearly_data.drop_duplicates(subset=["tin"], keep="last", inplace=True)
    yearly_data["year"] = year    
    yearly_data = yearly_data.merge(group_sizes, how="left", on="tin")
    yearly_data["confidence"] = 1 / yearly_data["size"]
    yearly_data.drop(columns=["start_date", "end_date", "size"], inplace=True)
    panel_elems.append(yearly_data)
panel = pd.concat(panel_elems)
panel.shape

(610553, 21)

In [79]:
panel.head(3)

,id,tin,reg_number,kind,category,first_name,last_name,patronymic,org_name,org_short_name,...,region,area,settlement,settlement_type,oktmo,lat,lon,address_raw,year,confidence
0,84075,5638061677,NaN,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭКСП...","ООО ""ЭКСПЕРТ ПЛЮС""",...,Оренбургская область,Оренбургский район,Зауральный,п,53634416101.0,51.7988888888889,54.9111111111111,ОРЕНБУРГСКАЯ / ОБЛАСТЬ / ОРЕНБУРГСКИЙ / РАЙОН ...,2016,1.0
1,85749,7715874151,1117746538535,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КОМП...","ООО ""КОМПАНИЯ Т.Т.Т. - АНАЛИТИК""",...,Москва,NaN,NaN,г,45000000.0,55.7540471,37.620405,МОСКВА / ГОРОД / / / / / /,2016,1.0
2,85748,7715772093,NaN,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЮРПР...","ООО ""ЮПК""",...,Москва,NaN,NaN,г,45000000.0,55.7540471,37.620405,МОСКВА / ГОРОД / / / / / /,2016,1.0


In [80]:
panel.to_csv("rsmp/csv/panel.csv", index=False)